In [ ]:
import requests
import json

baseURL = "https://api.coingecko.com/api/v3"
landing_folder = ""

# yg bakal dipake
# endpoint_coins_markets

endpoint_ping = "/ping"
endpoint_simple_price = "/simple/price"
endpoint_supported_vs_currencies = "/simple/supported_vs_currencies"

endpoint_coins_list = "/coins/list"
endpoint_coins_markets = "/coins/markets"
endpoint_coins_id = "/coins/"

endpoint_exchanges_list = "/exchanges/list"

params_simple_price = {
    "ids": "bitcoin",
    "vs_currencies": "idr",
}

params_coins_markets = {
    "vs_currency": "usd",
    "ids": "bitcoin,ethereum,binancecoin",
}

params_coins_id = {
    "id": "bitcoin",
    "tickers": "false",
}

In [ ]:
requestURL = baseURL + endpoint_coins_markets
print(requestURL)

r = requests.get(requestURL, params=params_coins_markets)

print(r.url)

pretty_json = json.loads(r.text)
print(json.dumps(pretty_json, indent=2))

# with open(landing_folder + "\\tes_dump.json", "w") as write_file:
#     json.dump(pretty_json, write_file, indent=2)